In [ ]:
import imagepers
import numpy as np
from read_chroma import read_chromato_and_chromato_cube

In [ ]:
file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'
mod_time = 1.7

In [ ]:
chromato_tic_preprocessed, time_rn, chromato_cube_preprocessed, sigma, mass_range = read_chromato_and_chromato_cube(file_path, mod_time, pre_process=True)


In [ ]:
persistances = np.array([p[2] for p in imagepers.persistence(chromato_cube_preprocessed)])
min_persistence = np.percentile(persistances, 90) / np.max(chromato_cube_preprocessed)


In [ ]:
persistances = np.array([p[2] for p in imagepers.persistence(chromato_cube_preprocessed)])
plt.hist(persistances, bins=100)
plt.axvline(min_persistence * np.max(chromato_cube_preprocessed), color='red', label='Dynamic threshold')
plt.legend()
plt.title("Distribution of peak persistence")
plt.show()